## Basic API Call to 2025.2 Version with API Key

In [7]:
import requests

# Base URL and endpoint
url = "https://crowdlabel.tii.ae/api/2025.2/tasks/pick"

# Headers with your API key
headers = {
    "x-api-key": "mvHvHQf-wne4WPIOksCELATVd9VLFegA"  # Replace with your actual key
}

# Optional query parameters
params = {
    "lang": "en",
    "category": "vqa"
}

# Sending the GET request
response = requests.get(url, headers=headers, params=params, verify=False)


# Handling the response
if response.status_code == 200:
    data = response.json()
    print("Data received:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Data received: [{'id': 'yhEgMNwFZkTYDC7L', 'category': 'vqa', 'complexity': 1, 'type': 'true-false', 'topic': 'living-room', 'content': {'image': {'filename': 'tii_vqa_0309tt3xh1qa148o.png', 'url': 'https://s3-eu-north-1-derc-wmi-crowdlabel-production.s3.eu-north-1.amazonaws.com/tii_vqa_0309tt3xh1qa148o.png'}}, 'task': {'text': 'is there a red metal thing of the same size as the gray rubber thing?', 'choices': [{'key': 'a', 'value': 'False'}, {'key': 'b', 'value': 'True'}]}, 'track_id': 't-hfMs0oXedLAdTD'}]


## Viewing Response

In [8]:
# Extract info
task = data[0]
question = task['task']['text']
choices = task['task']['choices']
image_url = task['content']['image']['url']

In [14]:
import numpy as np
import cv2
from IPython.display import Image, display, Markdown

# Display question
print(f"Question: {question}")
for choice in choices:
    print(f"{choice['key']}: {choice['value']}")

# format question and choices
question_md = f"### {question}\n" + "\n".join([f"- **{c['key']}**: {c['value']}" for c in choices])
display(Markdown(question_md))

# display image
display(Image(url=image_url))

Question: is there a red metal thing of the same size as the gray rubber thing?
a: False
b: True


### is there a red metal thing of the same size as the gray rubber thing?
- **a**: False
- **b**: True

## Building a dataframe of sample responses

In [17]:
import requests
import pandas as pd
import time

# Setup
url = "https://crowdlabel.tii.ae/api/2025.2/tasks/pick"
headers = {
    "x-api-key": "mvHvHQf-wne4WPIOksCELATVd9VLFegA"
}
params = {
    "lang": "en",
    "category": "vqa"
}

# Initialize
collected_ids = set()
records = []
max_iterations = 100
no_new_data_limit = 7
no_new_data_count = 0

for _ in range(max_iterations):
    response = requests.get(url, headers=headers, params=params, verify=False)
    
    if response.status_code == 200:
        data = response.json()
        if not data:
            print("Empty response received.")
            no_new_data_count += 1
        else:
            task = data[0]  # single task per call
            task_id = task["id"]
            if task_id not in collected_ids:
                collected_ids.add(task_id)
                no_new_data_count = 0  # reset on new data
                records.append({
                    "id": task_id,
                    "complexity": task["complexity"],
                    "type": task["type"],
                    "topic": task["topic"],
                    "image_url": task["content"]["image"]["url"],
                    "question": task["task"]["text"],
                    "choices": str(task["task"]["choices"])  # store as string
                })
            else:
                print(f"Duplicate task skipped: {task_id}")
                no_new_data_count += 1
    else:
        print(f"Request failed with status {response.status_code}")
        no_new_data_count += 1

    if no_new_data_count >= no_new_data_limit:
        print(f"Stopped early: No new data in last {no_new_data_limit} attempts.")
        break

    time.sleep(0.5)  # be kind to the API

# Save to CSV
df = pd.DataFrame(records)
df.to_csv("unique_tasks.csv", index=False)
print(f"Saved {len(df)} unique tasks to 'unique_tasks.csv'")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e

Duplicate task skipped: Xlxgk9zGbUHIouD4


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'crowdlabel.tii.ae'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e

: 

In [1]:
df.head()

NameError: name 'df' is not defined